In [9]:
import torch
import sys

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'MPS' if torch.backends.mps.is_available() else 'CPU'}")

Python: 3.11.14 | packaged by conda-forge | (main, Oct 22 2025, 22:56:31) [Clang 19.1.7 ]
PyTorch: 2.5.1
Device: MPS


In [10]:
import torch
import os
import sys
import time
import glob
import json
import shutil
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ultralytics import YOLO
from thop import profile
from pathlib import Path

# device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device='cpu'
print(f"Using device: {device}")

Using device: cpu


# YOLOv12 Training

Train YOLOv12n model on the dental X-ray dataset.


In [11]:
# Setup YOLOv12 path
project_root = Path('..').resolve() if Path.cwd().name == 'notebooks' else Path('.').resolve()
yolov12_repo_path = project_root / 'yolov12'
sys.path.insert(0, str(yolov12_repo_path))

print(f"✓ YOLOv12 repo: {yolov12_repo_path}")

✓ YOLOv12 repo: /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/yolov12


In [12]:
# Setup paths
processed = project_root / 'data' / 'processed'
models_dir = project_root / 'models'
models_dir.mkdir(parents=True, exist_ok=True)

# Locate dataset
data_yaml_path = processed / 'dataset.yaml'
if not data_yaml_path.exists():
    raise FileNotFoundError("Dataset not found! Run 01_prepare_dataset.ipynb first.")

data_yaml_path = str(data_yaml_path)
test_images_path = Path(data_yaml_path).parent / "test" / "images"

print(f"✓ Data config: {data_yaml_path}")
print(f"✓ Models dir: {models_dir}")

✓ Data config: /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/data/processed/dataset.yaml
✓ Models dir: /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/models


## Train Model


In [13]:
from ultralytics import YOLO
model_v12 = YOLO('yolov12n.pt')

# YOLOv12 has compatibility issues with MPS, use CPU instead
device_v12 = 'cpu' if device == 'mps' else device
if device == 'mps':
    print("⚠ YOLOv12 not compatible with MPS, using CPU instead")

print(f"Training YOLOv12n on {device_v12}...")
start_time_v12 = time.time()
results_v12 = model_v12.train(
    data=data_yaml_path,
    epochs=100,
    imgsz=640,
    batch=8,
    name='yolov12n_dental',
    device=device_v12,
    verbose=True,  # Shows progress bar with epochs
    plots=False,   # Disable plot generation
    save=True
)
train_time_v12 = time.time() - start_time_v12

path_v12_best_weights = results_v12.save_dir / 'weights' / 'best.pt'
print(f"\n✓ Training complete: {train_time_v12:.2f}s")
print(f"✓ Best weights: {path_v12_best_weights}")

Training YOLOv12n on cpu...
Ultralytics 8.3.226 🚀 Python-3.11.14 torch-2.5.1 CPU (Apple M4)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/data/processed/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov12n_dental3, nbs=64, nms=False, opset=None, optimi

/opt/anaconda3/envs/dental-xray/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/dental-xray/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <EB3FF92A-5EB1-3EE8-AF8B-5923C1265422> /opt/anaconda3/envs/dental-xray/lib/python3.11/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/dental-xray/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/dental-xray/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/dental-xray/lib/python3.11/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/dental-xray/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your e

optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 113 weight(decay=0.0), 120 weight(decay=0.0005), 119 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/notebooks/runs/detect/yolov12n_dental3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      1/100         0G      2.276      4.248      1.701        106        640: 100% ━━━━━━━━━━━━ 7/7 0.6it/s 12.0s1.4s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 2.6it/s 0.4s
                   all          6        172          0          0          0          0

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size

## Profile Model


In [19]:
print("Profiling model...")
model_v12_trained = YOLO(path_v12_best_weights)

# Move to CPU for THOP compatibility
model_cpu = model_v12_trained.model.cpu()
dummy_input = torch.randn(1, 3, 640, 640)

flops_v12, params_v12 = profile(model_cpu, inputs=(dummy_input,), verbose=False)
print(f"✓ Parameters: {params_v12/1e6:.2f}M | FLOPs: {flops_v12/1e9:.2f}G")

Profiling model...
✓ Parameters: 2.54M | FLOPs: 3.01G


## Measure Inference Time


In [20]:
print("Measuring inference time...")
# Reload model on correct device for inference
model_v12_trained = YOLO(path_v12_best_weights)

test_images = list(test_images_path.glob('*.jpg')) + list(test_images_path.glob('*.png'))

def get_avg_inference_time(model, image_list):
    if len(image_list) == 0:
        return 0
    _ = model(image_list[0], verbose=False)  # Warm-up
    total_time = 0
    for img in image_list:
        start = time.perf_counter()
        _ = model(img, verbose=False)
        total_time += time.perf_counter() - start
    return (total_time / len(image_list)) * 1000

avg_inf_time_v12 = get_avg_inference_time(model_v12_trained, test_images[:10])
print(f"✓ Avg inference: {avg_inf_time_v12:.2f}ms/image ({len(test_images)} test images)")

Measuring inference time...
✓ Avg inference: 64.93ms/image (7 test images)


In [21]:
print("Evaluating on test set...")
metrics_v12 = model_v12_trained.val(split='test', data=data_yaml_path, verbose=False)
map50_v12 = metrics_v12.box.map50
map50_95_v12 = metrics_v12.box.map

print(f"✓ mAP@50: {map50_v12:.4f} | mAP@50-95: {map50_95_v12:.4f}")

Evaluating on test set...
Ultralytics 8.3.226 🚀 Python-3.11.14 torch-2.5.1 CPU (Apple M4)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 6291.4±2028.1 MB/s, size: 931.4 KB)
val: Scanning /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/data/processed/test/labels.cache... 7 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 7/7 28.4Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 1/1 2.7it/s 0.4s
                   all          7        206      0.938      0.853      0.911      0.609
Speed: 0.3ms preprocess, 33.0ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/notebooks/runs/detect/val2
✓ mAP@50: 0.9107 | mAP@50-95: 0.6086


## Save Results


In [22]:
print("Saving model and metrics...")
shutil.copy(path_v12_best_weights, models_dir / 'yolov12_best.pt')

metrics_data = {
    'model': 'YOLOv12n',
    'training_time': train_time_v12,
    'params': params_v12 / 1e6,
    'inference_time_ms': avg_inf_time_v12,
    'map50': float(map50_v12),
    'map50_95': float(map50_95_v12)
}

with open(models_dir / 'yolov12_metrics.json', 'w') as f:
    json.dump(metrics_data, f, indent=2)

print(f"\n{'='*60}")
print(f"✓ Model saved: {models_dir}/yolov12_best.pt")
print(f"✓ Metrics saved: {models_dir}/yolov12_metrics.json")
print(f"{'='*60}")

Saving model and metrics...

✓ Model saved: /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/models/yolov12_best.pt
✓ Metrics saved: /Users/richa/Documents/Deep Learning Project/Dental-X-Ray-Cavity-Detection/models/yolov12_metrics.json


## Visualize Predictions


In [23]:
print("Visualizing sample predictions...")
sample_images = test_images[:3]
fig, axes = plt.subplots(1, len(sample_images), figsize=(15, 5))
if len(sample_images) == 1:
    axes = [axes]

for idx, img_path in enumerate(sample_images):
    results = model_v12_trained(img_path, verbose=False)
    plotted = results[0].plot()[..., ::-1]  # BGR to RGB
    axes[idx].imshow(plotted)
    axes[idx].axis('off')
    axes[idx].set_title(f"Sample {idx+1}")

plt.tight_layout()
plt.show()
print("\n✅ YOLOv12 training complete!")

Visualizing sample predictions...


<Figure size 1500x500 with 3 Axes>


✅ YOLOv12 training complete!
